In [1]:
!pip install faiss-cpu
!pip install annoy

  Using cached annoy-1.17.3.tar.gz (647 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for annoy
Failed to build annoy


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [24 lines of output]
      C:\Users\aksha\anaconda3\envs\sihenv\lib\site-packages\setuptools\__init__.py:85: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
      C:\Users\aksha\anaconda3\envs\sihenv\lib\site-packages\setuptools\_distutils\dist.py:268: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      running bdist_wheel
      running build
      running build_py
      creating buil

In [2]:
pip install -U langchain-community


  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ----------------------------- ---------- 1.8/2.5 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 8.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 9.6 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.22
    Uninstalling langchain-core-0.3.22:
      Successfully uninstalled langchain-core-0.3.22
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.

In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers

In [4]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

extracted_data = load_pdf(r"C:\\Users\\aksha\\OneDrive\\Desktop\\sih_cb\\Q and A.pdf")

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)

print(f"Number of chunks: {len(text_chunks)}")


Number of chunks: 67


In [6]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()

C:\Users\aksha\AppData\Local\Temp\ipykernel_2856\2261448754.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\aksha\anaconda3\envs\sihenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
docsearch = FAISS.from_texts([t.page_content for t in text_chunks], embeddings)

query = "What are allergies?"
docs = docsearch.similarity_search(query, k=3)
print("Top 3 Documents for the Query:", docs)

Top 3 Documents for the Query: [Document(metadata={}, page_content='A: Use the age filter on our resource pages to find articles, activities, and tips \ntailored to your child’s specific age group! \n56. Q: Is there a mobile app available for the website? 📱 \nA: Currently, we do not have a mobile app, but the website is mobile-friendly \nand can be easily accessed from your smartphone or tablet! \n57. Q: Can I access the website’s resources in multiple languages? 🌍 \nA: Yes! We offer select resources in multiple languages to accommodate'), Document(metadata={}, page_content='35. Q: What are some ways to teach my child about diversity and inclusion? \n🌍 \nA: Expose them to different cultures through books, films, and \ncommunity events, and discuss the importance of empathy and respect! \n36. Q: How can I help my child manage stress during busy school periods? \n😌 \nA: Teach them relaxation techniques, encourage physical activity, and \nensure they have time for hobbies and downtime! \n

In [8]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}

In [9]:
pip install CTransformers

Note: you may need to restart the kernel to use updated packages.


In [10]:
llm = CTransformers(
    model="TheBloke/llama-2-7b-chat-GGML",
    model_type="llama",
    config={'max_new_tokens': 512, 'temperature': 0.8}
)


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 998.41it/s]


In [11]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [35]:
print("\nHello! Welcome to the chatbot. Ask me any question, and I'll do my best to help you!")

while True:
    try:
        user_input = input("\nInput your question (type 'exit' to stop): ")

        if user_input.lower() == 'exit':
            print("\nThank you for using the chatbot! Have a great day!")
            break
        
        result = qa({"query": user_input})
        response = result["result"]
        
        print(f"\nUser Question: {user_input}")
        print(f"Bot Response: {response}\n")
    
    except KeyboardInterrupt:
        print("\nThank you for using the chatbot! Have a great day!")
        break



Hello! Welcome to the chatbot. Ask me any question, and I'll do my best to help you!


C:\Users\aksha\AppData\Local\Temp\ipykernel_2856\4107275646.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": user_input})



User Question: what is navshiksha?
Bot Response: Navshiksha is a free online platform that provides affordable quality education to underprivileged children in India. It was founded by two IIT Delhi alumni, who recognized the lack of access to quality education for disadvantaged children in rural areas. The platform offers a range of courses and activities aimed at improving their cognitive, emotional, and social development.

